In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

for p in ["..", "../..", "../../app"]:
    module_path = os.path.abspath(os.path.join(p))
    if module_path not in sys.path:
        sys.path.append(module_path)

import index

import app.elastic_query as elastic_query  # noqa: E402
from app import query_config

In [ ]:
from elasticsearch import Elasticsearch

# Connect to Elasticsearch
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "hammasir"),
    verify_certs=False,
    ssl_show_warn=False,
)
es.info()

In [ ]:
# data = index.get_dat("../data/processed/base_dataset.csv")
# index.index(data, reindex=False)

In [ ]:
# Example usage
import json

search_params = {
    "city": ["مشهد"],
    "gender": ["زن"],
    "expertise": ["ستون فقرات"],
    "problem": ["کمرم"],
    "neighborhood": ["کوهسنگی"],
    "online-or-in-person": ["hth"],
    "first-available-appointment": ["ww"],
    "moral": ["hth"],
}

config = elastic_query.configure_query(search_params, query_config.query_config)
final_query = elastic_query.build_query(search_params, config)
print(json.dumps(final_query, indent=2, ensure_ascii=False))
results = es.search(index="doctors", query=final_query)

for result in results["hits"]["hits"]:
    print(result)
    result = result["_source"]
    print(result["expertise"], result["star"])

print(len(results["hits"]["hits"]))

In [ ]:
explain = json.dumps(
    es.explain(
        index="doctors",
        id="14866",
        body={"query": final_query},
    ).body,
    indent=2,
    ensure_ascii=False,
)
print(
    explain,
)

In [ ]:
resp = es.indices.analyze(
    text="چشمام",
)
print(resp)